

# Motor imagery decoding from EEG data using the Common Spatial Pattern (CSP)

Decoding of motor imagery applied to EEG data decomposed using CSP. A classifier is then applied to features extracted on CSP-filtered signals.

### Imports

In [1]:
%matplotlib widget

import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score

from mne import Epochs, pick_types, events_from_annotations, set_log_level
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci

from my_CSP import MyCSP
from signal_power import Average_Power

### Loading data

In [30]:
set_log_level(verbose=False)

print('Loading Physionet recordings...', end='')

RIGHT_OR_LEFT_FIST=[1]
RIGHT_OR_LEFT_FIST=[2]
RIGHT_OR_LEFT_FIST=[6, 10, 14]
RIGHT_OR_LEFT_FIST_MI=[3, 7, 11]
FISTS_OR_FEET=[4, 8, 12]
FISTS_OR_FEET_MI=[5, 9, 13]

raw_fnames = eegbci.load_data(subject=1, runs=RIGHT_OR_LEFT_FIST)             # -Get paths to edf files.
raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames]) # -Load(read_raw_edf) in memomry(preloard=True) recprdings and convcatenate them.
eegbci.standardize(raw)  # set channel names                                # -Not sure what this does but it's necessary...
raw.set_montage(make_standard_montage("standard_1005"))                     # -Specify to MNE what montage/setup was used during the recording.
print(' done.')                                                             #  "standard_1005" refers to a standardized way of placing electrodes on the testee.
                                                                            #  See https://en.wikipedia.org/wiki/10%E2%80%9320_system_(EEG).

# raw.plot()
print(f'Applying band-pass filter...', end='')
raw.filter(7.0, 30.0, skip_by_annotation="edge")                            # -Apply bandpass filter, only keep frequecies in the range 7-30 Hz.
print(' done.')                                                             
# raw.plot()
              
events, _ = events_from_annotations(raw, event_id=dict(T1=1, T2=2))         # -Make events from annotaions, only use T1 and T2 annotations. Mark them as 0 and 1 respectively.
                                                                            #  According to the Physionet EEG-MI dataset web page(https://physionet.org/content/eegmmidb/1.0.0/),
                                                                            #  T0 corresponds to the motion of both fists and T1 to the motion of both feet.

print('Creating epochs from raw data...', end='')
picks = pick_types(raw.info, eeg=True)                                      # -Specify that we only want to listen to the EEG channels, the other channels are set to False by default.
# Testing will be done with a running classifier
epochs = Epochs(
    raw,
    events,
    tmin=-1.0,
    tmax=4.0,
    proj=True,
    picks=picks,
    baseline=None,
    preload=True,
)
print(' done.')

Loading Physionet recordings... done.
Applying band-pass filter... done.
Creating epochs from raw data... done.


### Training

In [31]:
epochs_data_train = epochs.get_data(tmin=1.0, tmax=2.0)
labels = epochs.events[:, 2]

cv = ShuffleSplit(10, test_size=0.2, random_state=42)
cv_splits_iterator = cv.split(epochs_data_train)

# Assemble the classifier
clf = Pipeline([("csp", MyCSP(n_components=2)), ("lda", LinearDiscriminantAnalysis())])

# Use scikit-learn Pipeline with cross_val_score function
scores = cross_val_score(clf, epochs_data_train, labels, cv=cv, n_jobs=None, error_score='raise')
print('Classification scores:', *scores, sep='\n')
print('Mean accuracy:', np.mean(scores))
_ = clf.fit(epochs_data_train, labels)

Classification scores:
0.8888888888888888
1.0
0.8888888888888888
1.0
1.0
0.8888888888888888
0.8888888888888888
1.0
0.8888888888888888
1.0
Mean accuracy: 0.9444444444444444


### Replaying EEG recordings in realtime

In [13]:
raw_data = raw.get_data()

# make stim data from events.
stim_data = np.empty((raw_data.shape[1]))
for i in range(events.shape[0] - 1):
    stim_data[events[i, 0]:events[i+1, 0]] = events[i, 2]
stim_data[events[-1, 0]:stim_data.shape[0]] = events[-1, 2]

window_size = epochs_data_train.shape[2]
for start in range(0, raw_data.shape[1]-window_size):
    data_window = [raw_data[:, start:start+window_size]]
    stim = stim_data[start + window_size]
    prediction = clf.predict(data_window)[0]
    print('start:', start,' stim:', stim, ' prediction:', prediction, ' ', 'Success' if prediction == stim else 'Failure')

start: 0  stim: 1.0  prediction: 2   Failure
start: 1  stim: 1.00625  prediction: 2   Failure
start: 2  stim: 1.0125  prediction: 2   Failure
start: 3  stim: 1.01875  prediction: 2   Failure
start: 4  stim: 1.025  prediction: 2   Failure
start: 5  stim: 1.03125  prediction: 2   Failure
start: 6  stim: 1.0375  prediction: 2   Failure
start: 7  stim: 1.04375  prediction: 2   Failure
start: 8  stim: 1.05  prediction: 2   Failure
start: 9  stim: 1.05625  prediction: 2   Failure
start: 10  stim: 1.0625  prediction: 2   Failure
start: 11  stim: 1.06875  prediction: 2   Failure
start: 12  stim: 1.075  prediction: 2   Failure
start: 13  stim: 1.08125  prediction: 2   Failure
start: 14  stim: 1.0875  prediction: 2   Failure
start: 15  stim: 1.09375  prediction: 2   Failure
start: 16  stim: 1.1  prediction: 2   Failure
start: 17  stim: 1.10625  prediction: 2   Failure
start: 18  stim: 1.1125  prediction: 2   Failure
start: 19  stim: 1.11875  prediction: 2   Failure
start: 20  stim: 1.125  predic

### References

See [CSP wikipedia page](https://en.wikipedia.org/wiki/Common_spatial_pattern) and  [Koles1991](https://doi.org/10.1016/0013-4694(91)90163-X).  
The EEGBCI dataset is documented in [SchalkEtAl2004](http://doi:10.1109/TBME.2004.827072)  and is available at [PhysioNet](https://physionet.org/content/eegmmidb/1.0.0/), [GoldbergerEtAl2000](https://doi.org/10.1161/01.CIR.101.23.e215).  

- Author: Mauro Abidal Carrer <mauroabidal@yahoo.fr>  
- Orginal Authors: Martin Billinger <martin.billinger@tugraz.at>  
- Original script: https://mne.tools/stable/auto_examples/decoding/decoding_csp_eeg.html  
- License: BSD-3-Clause  